In [ ]:
# 1. from 'CRF_2022-01-27_T1Close.ipynb'
# 2. removed unused cells (except pairplot), sequentially re-labeled cells
# 3. notebook functional

# Cell 1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import seaborn as sns

In [ ]:
# Import libraries

# Cell 2
import datetime
from datetime import datetime
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import plotly_express as px
#import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import time
import warnings
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
warnings.filterwarnings("ignore")

In [ ]:
# Cell 3
pd.set_option("display.max_rows", None, "display.max_columns", None)

load in data and check data type

In [ ]:
# Cell 4
df = pd.read_csv('C:/0-Ubi/S_Value_3.csv')
df = df.iloc[:,1:]
df = df.drop(61)
df.index = pd.RangeIndex(len(df.index))
df2 = df.copy()
df.shape, df.dtypes, df2.shape

In [ ]:
# Cell 5
df.head()

In [ ]:
# Cell 6
df.columns = df.columns.str.replace('[', '')
df.columns = df.columns.str.replace(']', '')

check multicolinearity

In [ ]:
# Cell 7
# correlation
cor = df.corr()
cor

In [ ]:
# Cell 8

sns.heatmap(cor)

In [ ]:
# Cell 9
df.drop(['Date',
 'Time',
] ,axis=1, inplace=True)

In [ ]:
# Cell 10
df.iloc[1:5,]

split data for train and test dataset

In [ ]:
# Cell 11
y = df.V_Close
x = df.drop(['V_C_target'], axis = 1)
xtrain, xtest, ytrain,ytest = train_test_split(x,y,test_size = 0.3)

In [ ]:
# Cell 12
train = df2.iloc[xtrain.index]
test = df2.iloc[xtest.index]

normalize continuous variables

In [ ]:
# Cell 13
scaler = StandardScaler()
xtrain.iloc[:,:] = scaler.fit_transform(xtrain.iloc[:,:])
xtest.iloc[:,:] = scaler.transform(xtest.iloc[:,:])
# Xiaoyan's 01-20 code: removed numbers inside square brackets
# still has the same caveats

models and comparison

In [ ]:
# Cell 14
# linear regression
lr = LinearRegression()
lr.fit(xtrain, ytrain)
lr_confidence = lr.score(xtest, ytest)
train['linear_regression'] = lr.predict(xtrain)
test['linear_regression']=lr.predict(xtest)
lr_mse = mean_squared_error(ytest, test['linear_regression'])
lr_confidence, lr_mse

In [ ]:
# Cell 15
# ranom forest
rf = RandomForestRegressor()
rf.fit(xtrain, ytrain)
rf_confidence = rf.score(xtest, ytest)
train['ranom_forest'] = rf.predict(xtrain)
test['ranom_forest']=rf.predict(xtest)
rf_mse = mean_squared_error(ytest, test['ranom_forest'])
rf_confidence,rf_mse

In [ ]:
# Cell 16
# ridge
rg = Ridge()
rg.fit(xtrain, ytrain)
rg_confidence = rg.score(xtest, ytest)
train['ridge'] = rg.predict(xtrain)
test['ridge']=rg.predict(xtest)
rg_mse = mean_squared_error(ytest, test['ridge'])
rg_confidence, rg_mse

In [ ]:
# Cell 17
# xgboost
# Import xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Create the training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Instantiate the XGBRegressor: xg_reg
xg_reg = xgb.XGBRegressor(objective="reg:linear", n_estimators=10, seed=123)

# Fit the regressor to the training set
xg_reg.fit(xtrain, ytrain)

# Predict the labels of the test set: preds
preds = xg_reg.predict(xtest)

# Compute the rmse: rmse
rmse = np.sqrt(mean_squared_error(ytest, preds))
print("RMSE: %f" % (rmse))
train['xgboost'] = xg_reg.predict(xtrain)
test['xgboost']=preds

XGB_confidence = xg_reg.score(xtest, ytest)
xgb_mse = mean_squared_error(ytest, test['xgboost'])
XGB_confidence,xgb_mse

In [ ]:
# Cell 18
# Convert the training and testing sets into DMatrixes: DM_train, DM_test
DM_train = xgb.DMatrix(data=xtrain,label=ytrain)
DM_test =  xgb.DMatrix(data=xtest,label=ytest)

# Create the parameter dictionary: params
params = {"booster":"gblinear", "objective":"reg:linear"}

# Train the model: xg_reg
xg_reg2 = xgb.train(params = params, dtrain=DM_train, num_boost_round=5)

# Predict the labels of the test set: preds
preds = xg_reg2.predict(DM_test)
train['xgboost_matric'] = xg_reg2.predict(DM_train)
test['xgboost_matric']=preds
# Compute and print the RMSE
rmse = np.sqrt(mean_squared_error(ytest,preds))
xgb_metric_mse = mean_squared_error(ytest, preds)
xbg_metric_confidence = 1- sum((ytest - preds)**2)/sum((ytest - ytest.mean())**2)
print("RMSE: %f" % (rmse))

print('Prediction: %.3f' % preds[0])
xbg_metric_confidence,xgb_metric_mse

In [ ]:
# Cell 19
# LGBM
from lightgbm import LGBMRegressor


# Create the training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
params = {
    'objective': 'regression',
    'linear_tree': True,
}
#LGBM = LGBMRegressor(**params)
LGBM = LGBMRegressor()

# Fit the regressor to the training set
LGBM.fit(xtrain, ytrain)
train['lgbm'] = LGBM.predict(xtrain)
test['lgbm']=LGBM.predict(xtest)
LGBM_confidence = LGBM.score(xtest, ytest)
LGBM_mse = mean_squared_error(ytest, test['lgbm'])
LGBM_confidence, LGBM_mse

In [ ]:
# Cell 20
# support vector machine
svr = SVR()
svr.fit(xtrain, ytrain)
train['svr'] = svr.predict(xtrain)
test['svr']=svr.predict(xtest)
svr_confidence = svr.score(xtest, ytest)
svr_mse = mean_squared_error(ytest, test['svr'])
svr_confidence, svr_mse

In [ ]:
# Cell 21
# Compare performence of each model

names = ['Linear Regression', 'Random Forest', 'Ridge', 'SVR', 'XGBoost', 'XGBoost_metric', 'LGBM']
columns = ['model', 'accuracy']
scores = [lr_confidence, rf_confidence, rg_confidence, svr_confidence, XGB_confidence, xbg_metric_confidence, LGBM_confidence]
alg_vs_score = pd.DataFrame([[x, y] for x, y in zip(names, scores)], columns = columns)

In [ ]:
# Cell 22
alg_vs_score

In [ ]:
# Cell 23
fig = px.bar(alg_vs_score, y='accuracy', x='model',
            title="Performance of Different Models", color="model", hover_name="accuracy",
             color_discrete_sequence=px.colors.qualitative.Pastel
             )

fig.show()

In [ ]:
# Cell 24
names = ['Linear Regression', 'Random Forest', 'Ridge', 'SVR', 'XGBoost', 'XGBoost_metric','LGBM']
columns_mse = ['model', 'mean_square_error']
scores_mse = [lr_mse, rf_mse, rg_mse, svr_mse, xgb_mse, xgb_metric_mse,LGBM_mse]
alg_vs_mse = pd.DataFrame([[x, y] for x, y in zip(names, scores_mse)], columns = columns_mse)

In [ ]:
# Cell 25
alg_vs_mse

In [ ]:
# Cell 26
fig_mse = px.bar(alg_vs_mse, y='mean_square_error', x='model',
            title="Performance of Different Models", color="model", hover_name="mean_square_error",
             color_discrete_sequence=px.colors.qualitative.Pastel
             )

fig_mse.show()

In [ ]:
# Cell 27
train.head()

In [ ]:
# Cell 28
train.corr().loc['V_C_target']

In [ ]:
# Cell 29
test.head()

In [ ]:
# Cell 30
# variable correlation with V_Close
test.corr().loc['V_C_target']

In [ ]:
# Cell 31
# predict new data
newdata = pd.read_csv('C:/0-Ubi/S_Value_4.csv')
newdata2 = newdata.copy()
newdata.columns = newdata.columns.str.replace('[', '')
newdata.columns = newdata.columns.str.replace(']', '')
newdata.drop(['DT_ID','Date',
 'Time',
] ,axis=1, inplace=True)
newy = newdata.V_Close
newx = newdata.drop(['V_C_target'], axis = 1)
newx.iloc[:,:] = scaler.fit_transform(newx.iloc[:,:])

In [ ]:
# Cell 32
newx.head()

In [ ]:
# Cell 33
newdata2['linear_regression']=lr.predict(newx)
newdata2['random_forest']=rf.predict(newx)
newdata2['ridge'] = rg.predict(newx)
newdata2['xgboost'] = xg_reg.predict(newx)
newdata2['xgboost_metric'] = xg_reg2.predict(xgb.DMatrix(data=newx,label=newy))
newdata2['lgbm'] = LGBM.predict(newx)
newdata2['svm'] = svr.predict(newx)

In [ ]:
# Cell 34
newdata2.head()

In [ ]:
# Cell 35
newdata2.corr().loc['V_C_target']

In [ ]:
# Cell 36
# close distribution in train, test, an newdata
plt.hist(train['V_C_target'],label='train')
plt.hist(test['V_C_target'],label='test')
plt.hist(newdata2['V_C_target'],label='newdata')
plt.legend(ncol=3)

In [ ]:
# Cell 37
train.describe()

In [ ]:
# Cell 38
newdata2.describe()

In [ ]:
# Cell 39
# save result
train.to_csv('C:/0-Ubi/train_S_Value_4.csv')
test.to_csv('C:/0-Ubi/test_S_Value_4.csv')
newdata2.to_csv('C:/0-Ubi/newdata_S_Value_4.csv')

********************End***********************

Multiple Run

In [ ]:
# cell 40
# run cell 1 to 11 before running this session
n_run = 100
newdata = pd.read_csv('C:/0-Ubi/S_Value_4.csv')
newdata3 = newdata.copy()
newdata.columns = newdata.columns.str.replace('[', '')
newdata.columns = newdata.columns.str.replace(']', '')
newdata.drop(['DT_ID','Date',
 'Time',
] ,axis=1, inplace=True)
newy = newdata.V_Close
newx_orig = newdata.drop(['V_C_target'], axis = 1)
for i in range(n_run):
    xtrain, xtest, ytrain,ytest = train_test_split(x,y,test_size = 0.3)
    scaler = StandardScaler()
    xtrain.iloc[:,:] = scaler.fit_transform(xtrain.iloc[:,:])
    xtest.iloc[:,:] = scaler.transform(xtest.iloc[:,:])
    newx = newx_orig.copy()
    newx.iloc[:,:] = scaler.transform(newx.iloc[:,:])
    # linear regression
    lr = LinearRegression()
    lr.fit(xtrain, ytrain)
    newdata3['linear_reg'+str(i)] = lr.predict(newx)
    #ranom forest
    rf = RandomForestRegressor()
    rf.fit(xtrain, ytrain)
    newdata3['rf'+str(i)] = rf.predict(newx)
    #rige
    rg = Ridge()
    rg.fit(xtrain, ytrain)
    newdata3['rg'+str(i)] = rg.predict(newx)
    #xgboost
    xg_reg = xgb.XGBRegressor(objective="reg:linear", n_estimators=10, seed=123)
    xg_reg.fit(xtrain, ytrain)
    newdata3['xg_reg'+str(i)] = xg_reg.predict(newx)
    #sgboost with matric transformation
    DM_train = xgb.DMatrix(data=xtrain,label=ytrain)
    DM_new =  xgb.DMatrix(data=newx,label=newy)
    params = {"booster":"gblinear", "objective":"reg:linear"}
    xg_reg2 = xgb.train(params = params, dtrain=DM_train, num_boost_round=5)
    newdata3['xg_regm'+str(i)] = xg_reg2.predict(DM_new)
    # LGBM
    LGBM = LGBMRegressor()
    LGBM.fit(xtrain, ytrain)
    newdata3['lgbm'+str(i)] = LGBM.predict(newx)
    # svm   
    svr = SVR()
    svr.fit(xtrain, ytrain)
    newdata3['svr'+str(i)] = svr.predict(newx)

# prediction average for each model
newdata3['linear_reg_avg']  = newdata3[[j for j in newdata3.columns if 'linear_reg' in j]].mean(axis=1)
newdata3['rf_avg']  = newdata3[[j for j in newdata3.columns if 'rf' in j]].mean(axis=1)
newdata3['xg_reg_avg']  = newdata3[[j for j in newdata3.columns if 'xg_reg' in j]].mean(axis=1)
newdata3['xg_regm_avg']  = newdata3[[j for j in newdata3.columns if 'xg_regm' in j]].mean(axis=1)
newdata3['lgbm_avg']  = newdata3[[j for j in newdata3.columns if 'lgbm' in j]].mean(axis=1)
newdata3['svr_avg']  = newdata3[[j for j in newdata3.columns if 'svr' in j]].mean(axis=1)    
    
    
    
    
    

In [ ]:
# cell 41
newdata3.head()

In [ ]:
# cell 42
newdata3[['V_C_target','linear_reg_avg','rf_avg','xg_reg_avg','xg_regm_avg','lgbm_avg','svr_avg']].corr()

In [ ]:
# cell 43
newdata3.to_csv('C:/0-Ubi/newdata_S_Value_4_prediction_avg.csv')

In [ ]:
# cell 44
# save only the preiction, in case file have too many colunmns
newdata3[['DT_ID','linear_reg_avg','rf_avg','xg_reg_avg','xg_regm_avg','lgbm_avg','svr_avg']].to_csv('C:/0-Ubi/newdata_S_Value_4_prediction_only_avg.csv')

In [ ]:
# cell 45
newdata3[['DT_ID','linear_reg_avg','rf_avg','xg_reg_avg','xg_regm_avg','lgbm_avg','svr_avg']]